# Aztec G-dDT Test Run

In [1]:
import sys
sys.path.append('../')

import pandas as pd
import os
import plotly.express as px
import seaborn as sns 
import matplotlib.pyplot as plt

import plotly.io as pio
pio.renderers.default = "png"

## Perform Simulation

In [ ]:
from aztec_gddt.experiment import test_run

# HACK in order to do sims in single mode
runs = 3
sim_dfs = []
for run in range(runs):
    sim_dfs.append(test_run().set_index('l1_blocks_passed').assign(run=run))
mc_traj_df = pd.concat(sim_dfs)

traj_df = mc_traj_df.query("run == 0 & subset == 0 & simulation == 0")
traj_df.columns

## Visualization over Model Features

In [ ]:
traj_df.head(5)

In [ ]:
fig = px.scatter(traj_df.reset_index(), 'l1_blocks_passed', 'l2_blocks_passed')
fig.update_layout(yaxis_range=[0, 10], xaxis_range=[0,  10 * 3], width=1000, height=500)
fig.show()

In [ ]:
y = traj_df.current_epoch.map(lambda x: len(x.slots))

fig = px.line(traj_df, y=y, labels={"x": 'timestep', 'y': 'Slot Count for current epoch'})
fig.show()

In [ ]:
y = traj_df.last_epoch.map(lambda x: len(x.prover_quotes))

fig = px.scatter(traj_df, y=y, labels={"x": 'timestep', 'y': 'Prover Quotes'})
fig.show()

In [ ]:
y = traj_df.last_epoch.map(lambda x: x.accepted_prover_quote)

fig = px.line(traj_df, y=y, labels={"x": 'timestep', 'y': 'Accepted Prover Quote Value'})
fig.show()

In [ ]:
y = traj_df.last_epoch.map(lambda x: x.finalized)

fig = px.line(traj_df, y=y, labels={"x": 'timestep', 'y': 'Pending Epoch Finalization Status'})
fig.show()

In [ ]:
y = traj_df.last_reward

fig = px.line(traj_df, y=y, labels={"x": 'timestep', 'y': 'Last Reward'})
fig.show()

In [ ]:
y = traj_df.last_reward_time_in_l1

fig = px.line(traj_df, y=y, labels={"x": 'timestep', 'y': 'Last Reward Time'})
fig.show()

In [ ]:
y = traj_df.current_epoch.map(lambda x: sum(s.tx_total_mana for s in x.slots))

fig = px.line(traj_df, y=y, labels={"x": 'timestep', 'y': 'Total mana on current epoch'})
fig.show()

In [ ]:
y = ['cumm_unproven_epochs', 'cumm_resolved_epochs', 'cumm_finalized_epochs']

fig = px.line(traj_df, y=y)
fig.show()

In [ ]:
y = ['cumm_empty_blocks', 'cumm_finalized_blocks']

fig = px.line(traj_df, y=y)
fig.show()

In [ ]:
y = ['cumm_mana_used_on_finalized_blocks']

fig = px.line(traj_df, y=y)
fig.show()

In [ ]:
y = ['market_price_juice_per_mana', 'base_fee']

fig = px.line(traj_df, y=y)
fig.show()

In [ ]:
x = 'l2_blocks_passed'
y = ['excess_mana']

fig = px.line(traj_df, x=x, y=y)
fig.show()

In [ ]:
x = 'l2_blocks_passed'
y = ['congestion_multiplier']

fig = px.line(traj_df, x=x, y=y)
fig.show()

In [ ]:

fig = px.line(traj_df, x='excess_mana', y='congestion_multiplier')
fig.update_traces(opacity=.4)
fig.show()

In [ ]:
x = 'timestep'
y = ['market_price_l1_gas', 'market_price_l1_blobgas',
      'oracle_price_l1_gas', 'oracle_price_l1_blobgas']

fig = px.line(traj_df.query('timestep < 100'), x=x, y=y)
fig.show()

In [ ]:
y = ['market_price_juice_per_mana', 'oracle_price_juice_per_mana']

fig = px.line(traj_df.query('timestep < 100'), x=x, y=y)
fig.show()

In [ ]:

N = 500
x = traj_df.l2_blocks_passed.head(N)
y = traj_df.current_epoch.map(lambda x: x.slots[-1].tx_total_mana).head(N)
c = traj_df.current_epoch.map(
    lambda x: x.slots[-1].has_proposal_on_network).head(N)

fig = px.scatter(
    x=x,
    y=y,
    color=c, labels={
        "x": "l2 blocks passed",
        "y": "total mana per block",
        "color": "has proposal on network"},
    width=800,
    height=400)
fig.show()

In [ ]:
x = []
y = []
for i, row in traj_df.head(300).iterrows():
    y_lst = [a.score for a in row.agents]
    y += y_lst
    x += len(y_lst) * [i]

plt.hist2d(x, y, bins=(300, 20))
plt.xlabel('L1 Blocks Passed')
plt.ylabel('Agent Scores')
plt.show()

## Visualization over PSUU Per-Trajectory Metrics

In [23]:
from aztec_gddt.psuu.metrics import *

In [ ]:
for label, fn in PER_TRAJECTORY_METRICS.items():
    print(f"{label}: {fn.__name__}")
    print(mc_traj_df.reset_index().groupby(['simulation', 'subset', 'run']).apply(fn, include_groups=False))
    print("---")